In [1]:
import pandas as pd
import numpy as np
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score,roc_curve
from functions import decile_analysis
print('********* Required Libraries Imported *********')

********* Required Libraries Imported *********


In [2]:
#load model developing data
model_data = pd.read_parquet('Leobrix_Release_BBILAP_Combined_Data.parquet.gzip')
print('********* Data loaded ********* ')
print(model_data.shape)
# Make sure no duplicates in the data
model_data =  model_data.drop_duplicates(subset=['unique_reference_number'])
model_data =  model_data.drop_duplicates(subset=['latest_feature_id'])
print(model_data.shape)

********* Data loaded ********* 
(478328, 2831)
(478328, 2831)


In [3]:
#Create New Target (Good in 60-15 but bad in 90-24)
model_data['G15_B24'] = model_data.apply(
   lambda row: 0 if (row['60_in_15mob'] == 0 and row['90_in_24mob'] == 0)
    else ( 1 if (row['60_in_15mob'] == 0 and row['90_in_24mob'] == 1)
    else 2),axis=1)

model_data= model_data[model_data['G15_B24']!=2]
print('********* New Target Prepared *********')

********* New Target Prepared *********


In [4]:
def map_weight(value):
    if (value >= '2022-01') and (value <= '2022-06'):
        return 2
    elif (value >= '2021-07') and (value <= '2021-12'):
        return 1
    elif (value >= '2021-01') and (value <= '2021-06'):
        return 0.5
    else:
        return 1

model_data['Weight_vintage'] = model_data['month_year'].apply(map_weight)


In [40]:
data = model_data.copy()
# take one copy for model development data
#some bank data have inf very rate but by safe replace 
data.replace([np.inf, -np.inf], np.nan, inplace=True)

In [41]:
#Split the time interval for model building and Validation of oot
model_df1 = data[(data['month_year']>='2021-01') & (data['month_year']<='2022-06')]
oot_df1= data[(data['month_year']>='2022-07') & (data['month_year']<='2022-09')]
print('********* Data Splitted *********')
print(model_df1.shape, oot_df1.shape)

********* Data Splitted *********
(34344, 2833) (10802, 2833)


In [42]:
features_list = [
'num_PERSONAL LOAN_enq_last_24M',
'num_CONSUMER LOAN_enq_last_9M',
'total_emi',
'num_of_loans_le_1lac_non_credit',
'num_CREDIT CARD_enq_last_3M',
'Utilization_all',
'Total_outstanding_Balance_CREDIT CARD_opened_last_6M',
'Age_of_oldest_open_account',
 'Utilization_CREDIT CARD_opened_last_36M',
'highest_sanction_amount_non_credit',
'Total_high_credit',
'num_PL_LE_1_Lac_opened_in_last_3M',
    
# banking
'cash_withdrawal_sum',
'balance_5th_day_M2',
'bounced_transactions_count',
'Min_Balance_Min_M1_M2_M3',
'num_credit_gt_salary',
'upi_trans_count',
'Max_Credit_Avg_M1_M2_M3',
'ratio_total_debit_to_total_credit_M3',
'ratio_6th_day_balance_to_2th_day_balance_M1',

# Reconcilated Features

'percentage_ever_delinquent',
'months_since_most_recent_delinquency_all',
'percentage_utilization_gt_75',
'No of enq in past 90days',
'Num_Enq_non_BANK_last_12M',
'Num_of_installment_loan_opened',

   

'g406s',
'dm206s',

#loan
'statedIncome',
'deviation_from_mean_income#last_30_Days',
'deviation_from_median_income/standard_deviation_income#last_365_Days',
'yearOfBirth_mean',
    
'month_year','Weight_vintage',
'G15_B24']

In [43]:

model_df = model_df1[features_list]
oot_df = oot_df1[features_list]

# Separate features (X) and target (y) for Model and OOT datasets
X_model = model_df.drop(columns=['month_year', 'G15_B24']) # Adjust 'target' to your actual target column name
y_model = model_df['G15_B24']

X_oot = oot_df.drop(columns=['month_year', 'G15_B24'])
y_oot = oot_df['G15_B24']

#Handling Null Values
for i in X_model.columns:
    X_model[i].fillna(-9999,inplace=True)

for i in X_oot.columns:
    X_oot[i].fillna(-9999,inplace=True)



/tmp/ipykernel_3163968/2552150070.py:13: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  X_model[i].fillna(-9999,inplace=True)
/tmp/ipykernel_3163968/2552150070.py:16: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try usi

In [44]:
X_train, X_test, y_train, y_test = train_test_split(X_model, y_model, test_size=0.40,random_state=42)

In [45]:
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((20606, 34), (13738, 34), (20606,), (13738,))

In [46]:
#Split Training data into 80% train and 20% eval
X_train_new, X_eval, y_train_new, y_eval = train_test_split(
    X_train, y_train, test_size=0.2, random_state =42
)
print(X_train_new.shape, X_eval.shape, X_test.shape)

(16484, 34) (4122, 34) (13738, 34)


In [47]:
#Get vinatge weight values 
train_weights = X_train_new['Weight_vintage'].values
eval_weights = X_eval['Weight_vintage'].values
test_weights = X_test['Weight_vintage'].values
oot_weights = X_oot['Weight_vintage'].values

In [48]:
X_train_new = X_train_new.drop(columns=['Weight_vintage'])
X_eval = X_eval.drop(columns=['Weight_vintage'])
X_test = X_test.drop(columns=['Weight_vintage'])
X_oot = X_oot.drop(columns=['Weight_vintage'])

In [49]:
# 2. **Prepare DMatrix for XGBoost**
dtrain = xgb.DMatrix(X_train_new, label=y_train_new, weight=train_weights)
deval = xgb.DMatrix(X_eval, label=y_eval,weight=eval_weights)
dtest = xgb.DMatrix(X_test, label=y_test)
doot = xgb.DMatrix(X_oot, label=y_oot)


In [58]:
params = {
    'objective': 'binary:logistic',
    'eval_metric': 'auc',
    'learning_rate': 0.01,
     'max_depth': 2,
    'subsample': 0.3,
    'colsample_bytree': 0.3,
    'lambda': 6,
    'alpha': 4.8,
    'gamma': 5.0,
    'random_state': 42
}

evals = [(dtrain, 'train'), (deval, 'eval')]
xgb_model = xgb.train(
    params,
    dtrain,
    num_boost_round= 300,
    early_stopping_rounds= 40,
    evals=evals,
    verbose_eval=10
)

X_train_pred = X_train_new.copy()
X_eval_pred = X_eval.copy()
X_test_pred = X_test.copy()
X_oot_pred = X_oot.copy()


X_train_pred['y_actual'] = y_train_new
X_train_pred['y_pred'] = xgb_model.predict(dtrain)
X_train_pred['LID'] = model_df1.loc[X_train_new.index,'latest_feature_id'].values

X_eval_pred['y_actual'] = y_eval
X_eval_pred['y_pred'] = xgb_model.predict(deval)
X_eval_pred['LID'] = model_df1.loc[X_eval.index,'latest_feature_id'].values

X_test_pred['y_actual'] = y_test
X_test_pred['y_pred'] = xgb_model.predict(dtest)
X_test_pred['LID'] = model_df1.loc[X_test.index,'latest_feature_id'].values

X_oot_pred['y_actual'] = y_oot
X_oot_pred['y_pred'] = xgb_model.predict(doot)
X_oot_pred['LID'] = oot_df1.loc[X_oot.index,'latest_feature_id'].values

[0]	train-auc:0.52728	eval-auc:0.53603
[10]	train-auc:0.62755	eval-auc:0.58862
[20]	train-auc:0.63368	eval-auc:0.60437
[30]	train-auc:0.63596	eval-auc:0.61549
[40]	train-auc:0.63949	eval-auc:0.61897
[50]	train-auc:0.64531	eval-auc:0.62139
[60]	train-auc:0.64703	eval-auc:0.61937
[70]	train-auc:0.64860	eval-auc:0.62036
[80]	train-auc:0.65054	eval-auc:0.61631
[87]	train-auc:0.65051	eval-auc:0.61691


In [59]:
# 6. **Run Decile Analysis**
train_summary, train_auc, train_gini, train_ks = decile_analysis(X_train_pred, 'y_pred', 'y_actual')
eval_summary, eval_auc, eval_gini, eval_ks = decile_analysis(X_eval_pred, 'y_pred', 'y_actual')

test_summary, test_auc, test_gini, test_ks = decile_analysis(X_test_pred, 'y_pred', 'y_actual')
oot_summary, oot_auc, oot_gini, oot_ks = decile_analysis(X_oot_pred, 'y_pred', 'y_actual')

# 7. **Print Results**
print(f"Train AUC: {train_auc:.2f}, Gini: {train_gini:.2f}, KS: {train_ks:.2f}")
print(f"Eval AUC: {eval_auc:.2f}, Gini: {eval_gini:.2f}, KS: {eval_ks:.2f}")
print(f"Test AUC: {test_auc:.2f}, Gini: {test_gini:.2f}, KS: {test_ks:.2f}")
print(f"OOT AUC: {oot_auc:.2f}, Gini: {oot_gini:.2f}, KS: {oot_ks:.2f}")


Train AUC: 0.65, Gini: 0.29, KS: 0.21
Eval AUC: 0.61, Gini: 0.23, KS: 0.18
Test AUC: 0.62, Gini: 0.23, KS: 0.19
OOT AUC: 0.62, Gini: 0.25, KS: 0.18


In [60]:
train_summary

mean_actual  mean_prediction  max_prediction  min_prediction  count  \
Decile                                                                        
10         0.149181         0.126857        0.158664        0.119504   1649   
9          0.091019         0.115180        0.119498        0.111280   1648   
8          0.074029         0.107757        0.111279        0.104676   1648   
7          0.070952         0.102525        0.104670        0.100470   1649   
6          0.063714         0.099074        0.100468        0.097948   1648   
5          0.056501         0.097274        0.097948        0.096748   1646   
4          0.043030         0.096382        0.096748        0.096057   1650   
3          0.042527         0.095821        0.096055        0.095575   1599   
2          0.037190         0.095361        0.095575        0.095094   1694   
1          0.026618         0.094786        0.095089        0.094456   1653   

        label_good  label_bad  
Decile                         
10            1403        246  
9             1498        150  
8             1526        122  
7             1532        117  
6             1543        105  
5             1553         93  
4             1579         71  
3             1531         68  
2             1631         63  
1             1609         44

In [61]:
test_summary

mean_actual  mean_prediction  max_prediction  min_prediction  count  \
Decile                                                                        
10         0.118632         0.127063        0.158888        0.119988   1374   
9          0.085153         0.115469        0.119987        0.111479   1374   
8          0.081514         0.107916        0.111468        0.104845   1374   
7          0.061180         0.102533        0.104845        0.100541   1373   
6          0.066958         0.099155        0.100538        0.098015   1374   
5          0.042213         0.097350        0.098015        0.096787   1374   
4          0.035011         0.096443        0.096786        0.096128   1371   
3          0.040757         0.095847        0.096127        0.095588   1374   
2          0.051749         0.095373        0.095588        0.095125   1372   
1          0.029753         0.094814        0.095123        0.094456   1378   

        label_good  label_bad  
Decile                         
10            1211        163  
9             1257        117  
8             1262        112  
7             1289         84  
6             1282         92  
5             1316         58  
4             1323         48  
3             1318         56  
2             1301         71  
1             1337         41

In [62]:
oot_summary

mean_actual  mean_prediction  max_prediction  min_prediction  count  \
Decile                                                                        
10         0.153562         0.128309        0.155854        0.120766   1081   
9          0.102778         0.116323        0.120763        0.111933   1080   
8          0.084259         0.108334        0.111916        0.105145   1080   
7          0.077778         0.102856        0.105145        0.100869   1080   
6          0.078777         0.099404        0.100869        0.098193   1079   
5          0.074006         0.097484        0.098191        0.096944   1081   
4          0.052925         0.096525        0.096943        0.096178   1077   
3          0.057301         0.095902        0.096177        0.095632   1082   
2          0.037178         0.095421        0.095631        0.095171   1049   
1          0.042228         0.094840        0.095170        0.094456   1113   

        label_good  label_bad  
Decile                         
10             915        166  
9              969        111  
8              989         91  
7              996         84  
6              994         85  
5             1001         80  
4             1020         57  
3             1020         62  
2             1010         39  
1             1066         47

In [19]:
feature_importance = xgb_model.get_score(importance_type='weight')
feature_importance_df = pd.DataFrame(
    list(feature_importance.items()),
    columns =['Feature','Importance']
).sort_values(by='Importance',ascending=False)
print(feature_importance_df)

                                              Feature  Importance
25                                              g406s        79.0
23                          Num_Enq_non_BANK_last_12M        65.0
13                         bounced_transactions_count        33.0
22                           No of enq in past 90days        24.0
16                                    upi_trans_count        19.0
7                          Age_of_oldest_open_account        18.0
12                                cash_withdrawal_sum        17.0
5                                     Utilization_all        16.0
0                      num_PERSONAL LOAN_enq_last_24M        16.0
1                       num_CONSUMER LOAN_enq_last_9M        16.0
15                               num_credit_gt_salary        16.0
6   Total_outstanding_Balance_CREDIT CARD_opened_l...        16.0
27                                       statedIncome        13.0
28            deviation_from_mean_income#last_30_Days        12.0
21        

In [65]:
#Save the Pred
Train = X_train_pred[['LID','y_actual','y_pred']]
Eval = X_eval_pred[['LID','y_actual','y_pred']]
Test = X_test_pred[['LID','y_actual','y_pred']]
Oot = X_oot_pred[['LID','y_actual','y_pred']]
Train.to_csv('LeoPayu_BBILA_Train_Pred_1524_Vin.csv',index=False)
Eval.to_csv('LeoPayu_BBILA_Eval_Pred_1524_Vin.csv',index=False)
Test.to_csv('LeoPayu_BBILA_Test_Pred_1524_Vin.csv',index=False)
Oot.to_csv('LeoPayu_BBILA_Oot_Pred_1524_Vin.csv',index=False)


In [64]:
#Save the Model
xgb_model.save_model('LeoPayu_BBILA_15_in_24mob_Vin.json')
print('Model Saved')

Model Saved
